In [1]:
from config import running_backend, default_container  # noqa
from servicex_test_utils import wait_for_request_done, get_servicex_request_data
import requests
from servicex import ServiceXDataset
from servicex.minio_adaptor import MinioAdaptor
from servicex.servicex_adaptor import ServiceXAdaptor
import servicex
from func_adl_xAOD import ServiceXDatasetSource
import uproot_methods
from numpy import genfromtxt
import numpy as np

print("All modules imported.")

All modules imported.


In [14]:
# This can take a very long time - 15-30 minutes depending on the quality of your connection.
# If it is taking too long, most likely the problem is is the downloading - so look at the log
# from the rucio downloader to track progress (yes, an obvious feature request).
def test_func_adl_query_simple_jets(running_backend):
    query = "(call ResultTTree (call Select (call SelectMany (call EventDataset (list 'localds:bogus')) (lambda (list e) (call (attr e 'Jets') 'AntiKt4EMTopoJets'))) (lambda (list j) (/ (call (attr j 'pt')) 1000.0))) (list 'JetPt') 'analysis' 'junk.root')"
    dataset = "mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00"
    r = servicex.get_data(query , dataset, running_backend)
    print(len(r.index))
    return r

In [15]:
array = test_func_adl_query_simple_jets("http://localhost:5000/servicex")

11355980


In [16]:
print(len(array.index))

11355980


In [23]:
def test_func_adl_query_electrons_and_muons (running_backend):
    query = "(call ResultTTree (call Select (call Select (call EventDataset (list 'localds:bogus')) (lambda (list e) (list (call (attr e 'Electrons') 'Electrons') (call (attr e 'Muons') 'Muons')))) (lambda (list e) (list (call (attr (subscript e 0) 'Select') (lambda (list ele) (call (attr ele 'e')))) (call (attr (subscript e 0) 'Select') (lambda (list ele) (call (attr ele 'pt')))) (call (attr (subscript e 0) 'Select') (lambda (list ele) (call (attr ele 'phi')))) (call (attr (subscript e 0) 'Select') (lambda (list ele) (call (attr ele 'eta')))) (call (attr (subscript e 1) 'Select') (lambda (list mu) (call (attr mu 'e')))) (call (attr (subscript e 1) 'Select') (lambda (list mu) (call (attr mu 'pt')))) (call (attr (subscript e 1) 'Select') (lambda (list mu) (call (attr mu 'phi')))) (call (attr (subscript e 1) 'Select') (lambda (list mu) (call (attr mu 'eta'))))))) (list 'e_E' 'e_pt' 'e_phi' 'e_eta' 'mu_E' 'mu_pt' 'mu_phi' 'mu_eta') 'forkme' 'dude.root')"
    dataset = "mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00",
    r = servicex.get_data(query, dataset, data_type = "awkward", servicex_endpoint = running_backend)
    return r

In [24]:
emu = test_func_adl_query_electrons_and_muons("http://localhost:5000/servicex")

In [25]:
print(emu)

{b'e_E': <JaggedArray [[11678.290453057432] [124352.58759560794 5018.021807808239 28260.79313076634 4866.388771162408] [] ... [167994.90017338746 25607.67724485566 25950.88956615245 ... 16041.8821671694 6253.206926476292 16910.816994632994] [45230.94004367999 41885.42415619056 8183.519427098758] [101387.70455256793 127098.75170143174 3860.0051272222413 18531.43010673125]] at 0x01709519bc08>, b'e_pt': <JaggedArray [[5794.58154296875] [36922.66796875 4514.7587890625 4671.44482421875 3811.003662109375] [] ... [36469.15625 25503.25 13456.806640625 ... 4272.71728515625 4177.89990234375 3152.95556640625] [43632.55859375 39205.66796875 3786.251953125] [39072.98828125 43704.296875 3854.654541015625 3911.02734375]] at 0x01709519b848>, b'e_phi': <JaggedArray [[-0.7931161522865295] [2.266842842102051 0.4230227470397949 -0.7722882032394409 3.0246007442474365] [] ... [-2.1718931198120117 -2.48415470123291 2.543349504470825 ... -0.42512843012809753 0.5396857857704163 0.6070034503936768] [-2.51661348

In [5]:
def test_func_adl_query_simple_jets_large_sample(running_backend):
    'Do not run this unless you have a large well connected system at your beck and call!!'

    query = "(call ResultTTree (call Select (call SelectMany (call EventDataset (list 'localds:bogus')) (lambda (list e) (call (attr e 'Jets') 'AntiKt4EMTopoJets'))) (lambda (list j) (/ (call (attr j 'pt')) 1000.0))) (list 'JetPt') 'analysis' 'junk.root')"
    dataset = "data17_13TeV:data17_13TeV.periodK.physics_Main.PhysCont.DAOD_STDM7.grp22_v01_p3713"
    r = servicex.get_data(query, dataset, servicex_endpoint = running_backend)
    print(r)

print("Large jet tests defined.")

Large jet tests defined.


In [ ]:
test_func_adl_query_simple_jets_large_sample("http://localhost:5000/servicex")

In [9]:
def test_servicex_stress_capacity():
    dataset = ServiceXDataset('data17_13TeV:data17_13TeV.periodK.physics_Main.PhysCont.DAOD_STDM7.grp23_v01_p4030', max_workers = 100)
#    dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.DAOD_STDM3.e3601_e5984_s3126_r10201_r10210_p3975_tid20425969_00', max_workers = 400)
    query = ServiceXDatasetSource(dataset) \
        .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
        .Where('lambda j: (j.pt()/1000)>30') \
        .Select('lambda j: (j.pt())') \
        .AsPandasDF("JetPt") \
        .value()
    
    return query

In [10]:
stress_test = test_servicex_stress_capacity()

ServiceXException: (ServiceXException(...), 'Failed to transform all files')

In [4]:
print(stress_test)

               JetPt
entry               
0       33307.625000
1       53654.441406
2       51195.527344
3       72874.007812
4       60747.136719
...              ...
267581  44438.222656
267582  44139.707031
267583  34273.828125
267584  69459.250000
267585  44022.546875

[19452614 rows x 1 columns]


In [24]:
def test_func_adl_simple_jet_pts():
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")
    query = ServiceXDatasetSource(dataset) \
        .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
        .Where('lambda j: (j.pt()/1000)>30') \
        .Select('lambda j: (j.pt())') \
        .AsPandasDF("JetPt") \
        .value()
    
    return query

In [25]:
jet_test = test_func_adl_simple_jet_pts()